# 📊 Projeto: Superlotação de Leitos Hospitalares

Este notebook demonstra como realizar uma **previsão simples da Taxa de Ocupação Hospitalar** usando **Python, Pandas e Scikit-Learn**.

**Como usar no Colab:**
1) Faça upload do seu CSV `dados_hospitalares_corrigidos.csv` (ou ajuste o nome na célula de leitura)
2) Execute as células em ordem (Runtime → Run all)
3) Veja o gráfico com histórico + previsão para 6 meses


In [ ]:
# 1) Instalar/Importar bibliotecas (no Colab, instale se necessário)
!pip -q install pandas scikit-learn matplotlib

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
# 2) Carregar dados
from google.colab import files
print('📥 Selecione o arquivo CSV (ex.: dados_hospitalares_corrigidos.csv)')
uploaded = files.upload()

# Nome padrão (ajuste se o seu arquivo tiver outro nome)
csv_name = list(uploaded.keys())[0]
df = pd.read_csv(csv_name)
print('\n✅ CSV carregado:', csv_name)
df.head()

In [ ]:
# 3) Pré-processamento
df['AnoMes'] = pd.to_datetime(df['AnoMes'], errors='coerce')
df['Mes_num'] = df['AnoMes'].dt.year * 12 + df['AnoMes'].dt.month

# Conferir tipos
print(df.dtypes)
df.describe(include='all')

In [ ]:
# 4) Treinar modelo simples (Regressão Linear)
X = df[['Mes_num', 'Internacoes']]
y = df['Taxa_Ocupacao']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = LinearRegression()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE do modelo: {rmse:.2f}')

In [ ]:
# 5) Previsão para os próximos 6 meses
ultimo_mes = df['Mes_num'].max()
media_internacoes = df['Internacoes'].mean()
novos_meses = pd.DataFrame({
    'Mes_num': [ultimo_mes + i for i in range(1, 7)],
    'Internacoes': [media_internacoes] * 6
})
prev = modelo.predict(novos_meses)
datas_prev = pd.date_range(df['AnoMes'].max(), periods=6, freq='M')

# 6) Visualização (histórico + previsão)
plt.figure(figsize=(12,6))
plt.plot(df['AnoMes'], df['Taxa_Ocupacao'], label='Histórico')
plt.plot(datas_prev, prev, label='Previsão (6 meses)', linestyle='--')
plt.axhline(80, linestyle=':', label='Nível Atenção (80%)')
plt.axhline(90, linestyle=':', label='Nível Crítico (90%)')
plt.xlabel('Período')
plt.ylabel('Taxa de Ocupação (%)')
plt.title('📈 Previsão da Ocupação Hospitalar (6 meses)')
plt.legend()
plt.show()

### Observações
- Este é um **modelo simples** (Regressão Linear) para fins didáticos.
- Você pode melhorar incluindo variáveis como **leitos disponíveis, sazonalidade, SRAG (SIVEP-Gripe)** e outras fontes.
- Para usar no Looker, exporte as previsões como CSV e conecte como uma nova fonte.